**Importing the libraries needed**

In [1]:
import kagglehub
import shutil
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
! pip install onnx
! pip install onnxruntime
!pip install onnx2torch

! pip install onnxruntime-gpu
import onnx
from onnx2torch import convert
import onnx
from onnx2torch import convert
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
# Install correct versions
!pip install --quiet onnx onnx2torch torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Restart runtime after this cell before continuing



**Downloading the original dateset from Kaggle API**

In [2]:


# Downloading latest version of VGGFace2 dataset from KaggleHub
path = kagglehub.dataset_download("hearfool/vggface2")
print("Downloaded to:", path)

# Moving a  dataset to Colab-visible directory
visible_path = '/content/vggface2'

if not os.path.exists(visible_path):
    shutil.copytree(path, visible_path)
    print(f"Copied dataset to {visible_path}")
else:
    print(f"Dataset already exists at {visible_path}")






Downloaded to: /kaggle/input/vggface2
Copied dataset to /content/vggface2


**Eliminating train/val Folders to Retain Only Class-Based Image Folders**

In [3]:
import os
import shutil

# Set your base directory (the one containing 'train' and 'val')
base_dir = '/content/vggface2/'  # or set manually like '/path/to/vggface2'
print(base_dir)
for folder in ['train', 'val']:
    folder_path = os.path.join(base_dir, folder)

    for item in os.listdir(folder_path):
        src = os.path.join(folder_path, item)
        dst = os.path.join(base_dir, item)
        shutil.move(src, dst)
        print(f"Moved '{src}' to '{dst}'")

    # Remove the empty folder after moving its contents
    shutil.rmtree(folder_path)
    print(f"Removed folder '{folder_path}'")


/content/vggface2/
Moved '/content/vggface2/train/n000161' to '/content/vggface2/n000161'
Moved '/content/vggface2/train/n000266' to '/content/vggface2/n000266'
Moved '/content/vggface2/train/n000042' to '/content/vggface2/n000042'
Moved '/content/vggface2/train/n000297' to '/content/vggface2/n000297'
Moved '/content/vggface2/train/n000073' to '/content/vggface2/n000073'
Moved '/content/vggface2/train/n000442' to '/content/vggface2/n000442'
Moved '/content/vggface2/train/n000162' to '/content/vggface2/n000162'
Moved '/content/vggface2/train/n000494' to '/content/vggface2/n000494'
Moved '/content/vggface2/train/n000165' to '/content/vggface2/n000165'
Moved '/content/vggface2/train/n000186' to '/content/vggface2/n000186'
Moved '/content/vggface2/train/n000491' to '/content/vggface2/n000491'
Moved '/content/vggface2/train/n000152' to '/content/vggface2/n000152'
Moved '/content/vggface2/train/n000397' to '/content/vggface2/n000397'
Moved '/content/vggface2/train/n000131' to '/content/vggfa

**Counting number of folders and images files in the original data source**

In [4]:
# Count number of folders (classes) and image files
folder_count = 0
file_count = 0

for root, dirs, files in os.walk(visible_path):

    # Only count folders that directly contain images
    if any(file.lower().endswith(('.jpg')) for file in files):
        folder_count += 1
        file_count += len(files)

print(f"Number of folders: {folder_count}")
print(f"Total number of image files: {file_count}")


Number of folders: 540
Total number of image files: 197693


**Renaming the images with their folder they belonged to( representing the individuals followed by + and  the name of the file) for better tracking of the files and storing into a dictionary the new name of the images and their respective folder(representing a specific individual)**

In [5]:
import os

base_dir = '/content/vggface2'  # Change this to your folder path

filename_to_folder = {}

for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            new_name = f"{folder_name}+{filename}"
            new_path = os.path.join(folder_path, new_name)

            os.rename(file_path, new_path)
            filename_to_folder[new_name] = folder_name
            print(f"Renamed {filename} to {new_name}")

# Example: print dictionary entries
print("\nSample first 5 entries of the dictionary having as keys the name of the images and as values the number the folder they belong :")
for k, v in list(filename_to_folder.items())[:5]:
    print(f"{k} -> {v}")



Streaming output truncated to the last 5000 lines.
Renamed 0096_10.jpg to n000069+0096_10.jpg
Renamed 0155_01.jpg to n000069+0155_01.jpg
Renamed 0473_11.jpg to n000069+0473_11.jpg
Renamed 0145_01.jpg to n000069+0145_01.jpg
Renamed 0007_01.jpg to n000069+0007_01.jpg
Renamed 0170_03.jpg to n000069+0170_03.jpg
Renamed 0428_01.jpg to n000069+0428_01.jpg
Renamed 0071_01.jpg to n000069+0071_01.jpg
Renamed 0318_04.jpg to n000069+0318_04.jpg
Renamed 0017_01.jpg to n000069+0017_01.jpg
Renamed 0018_05.jpg to n000069+0018_05.jpg
Renamed 0250_02.jpg to n000069+0250_02.jpg
Renamed 0131_02.jpg to n000069+0131_02.jpg
Renamed 0086_01.jpg to n000069+0086_01.jpg
Renamed 0040_02.jpg to n000069+0040_02.jpg
Renamed 0424_01.jpg to n000069+0424_01.jpg
Renamed 0171_01.jpg to n000069+0171_01.jpg
Renamed 0430_01.jpg to n000069+0430_01.jpg
Renamed 0376_01.jpg to n000069+0376_01.jpg
Renamed 0243_01.jpg to n000069+0243_01.jpg
Renamed 0204_01.jpg to n000069+0204_01.jpg
Renamed 0268_02.jpg to n000069+0268_02.jpg
Ren

**Save folder of the dataset modified in Google Drive**

**Preprocessing the data**

In [2]:
# Preprocessing steps
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



**Using a data loader for the dataset**

In [4]:
# Loading dataset
visible_path = '/content/vggface2'

dataset = datasets.ImageFolder(root=visible_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Previewing one batch
for images, labels in dataloader:
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    break


Image batch shape: torch.Size([32, 3, 112, 112])
Label batch shape: torch.Size([32])


In [5]:
print(dataset.classes)# List of class folder names
num_classes=len(dataset.classes)
print('Number of classes is: ',num_classes)


['n000001', 'n000002', 'n000003', 'n000004', 'n000005', 'n000006', 'n000007', 'n000008', 'n000009', 'n000010', 'n000011', 'n000012', 'n000013', 'n000014', 'n000015', 'n000016', 'n000017', 'n000018', 'n000019', 'n000020', 'n000021', 'n000022', 'n000023', 'n000024', 'n000025', 'n000026', 'n000027', 'n000028', 'n000029', 'n000030', 'n000031', 'n000032', 'n000033', 'n000034', 'n000035', 'n000036', 'n000037', 'n000038', 'n000039', 'n000040', 'n000041', 'n000042', 'n000043', 'n000044', 'n000045', 'n000046', 'n000047', 'n000048', 'n000049', 'n000050', 'n000051', 'n000052', 'n000053', 'n000054', 'n000055', 'n000056', 'n000057', 'n000058', 'n000059', 'n000060', 'n000061', 'n000062', 'n000063', 'n000064', 'n000065', 'n000066', 'n000067', 'n000068', 'n000069', 'n000070', 'n000071', 'n000072', 'n000073', 'n000074', 'n000075', 'n000076', 'n000077', 'n000078', 'n000079', 'n000080', 'n000081', 'n000082', 'n000083', 'n000084', 'n000085', 'n000086', 'n000087', 'n000088', 'n000089', 'n000090', 'n000091'

In [6]:
from torch.utils.data import random_split, DataLoader

# Define split sizes
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [7]:
for images, labels in test_loader:
    print("Test Image batch shape:", images.shape)
    print("Test Label batch shape:", labels.shape)
    break


Test Image batch shape: torch.Size([32, 3, 112, 112])
Test Label batch shape: torch.Size([32])


**Setup device (CPU or GPU)**

In [8]:
#  Setup device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
torch.cuda.memory_summary(device=device)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [10]:
import gc
gc.collect()
torch.cuda.empty_cache()


**Installing and importing libraries needed to load and use pretrained model in .onnx format**

**Implementation of ArcFace Margin Layer for the losss( references https://github.com/HaiyuWu/SOTA-Face-Recognition-Train-and-Test/blob/main/losses.py)**

In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd

class ArcFace(nn.Module):
    def __init__(self, feature_in=512, feature_out=540, margin=0.3, scale = 64):
        super().__init__()
        self.feature_in = feature_in
        self.feature_out = feature_out
        self.scale = scale
        self.margin = margin

        self.weights = nn.Parameter(torch.FloatTensor(feature_out,feature_in))
        nn.init.xavier_normal_(self.weights)
    def forward(self, features, targets):
        cos_theta = F.linear(features, F.normalize(self.weights), bias=None)
        cos_theta = cos_theta.clip(-1+1e-7, 1-1e-7)

        arc_cos = torch.acos(cos_theta)
        M = F.one_hot(targets, num_classes = self.feature_out) * self.margin
        arc_cos += M

        cos_theta_2 = torch.cos(arc_cos)
        logits = cos_theta_2 * self.scale
        return logits


**Converting the onnnx model into a Pytorch model for fine-tuning purposes**

In [59]:
# Load ONNX model
onnx_model_path = "glint360k_r50.onnx"
onnx_model = onnx.load(onnx_model_path)

# Convert to PyTorch model
torch_model = convert(onnx_model)

for name, module in torch_model.named_modules():
    print(f"{name}: {module}")




: GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initializers): Module()
  (PRelu_1): OnnxPReLU()
  (BatchNormalization_2): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_4): OnnxPReLU()
  (Conv_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Conv_6): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
  (Add_7): OnnxBinaryMathOperation()
  (BatchNormalization_8): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_10): OnnxPReLU()
  (Conv_11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Add_12): OnnxBinaryMathOperation()
  (BatchNormalization_13): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=

In [60]:
import torch
dummy_input = torch.randn(2, 3, 112, 112)  # Adjust shape if needed
output = torch_model(dummy_input)
print(output.shape)  # Should be [1, 512]


torch.Size([2, 512])


In [61]:
for name, module in torch_model.named_modules():
    if hasattr(module, 'out_features'):
        print(f"{name}: {module.out_features}")
    elif hasattr(module, 'out_channels'):
        print(f"{name}: {module.out_channels}")


Conv_0: 64
Conv_3: 64
Conv_5: 64
Conv_6: 64
Conv_9: 64
Conv_11: 64
Conv_14: 64
Conv_16: 64
Conv_19: 128
Conv_21: 128
Conv_22: 128
Conv_25: 128
Conv_27: 128
Conv_30: 128
Conv_32: 128
Conv_35: 128
Conv_37: 128
Conv_40: 256
Conv_42: 256
Conv_43: 256
Conv_46: 256
Conv_48: 256
Conv_51: 256
Conv_53: 256
Conv_56: 256
Conv_58: 256
Conv_61: 256
Conv_63: 256
Conv_66: 256
Conv_68: 256
Conv_71: 256
Conv_73: 256
Conv_76: 256
Conv_78: 256
Conv_81: 256
Conv_83: 256
Conv_86: 256
Conv_88: 256
Conv_91: 256
Conv_93: 256
Conv_96: 256
Conv_98: 256
Conv_101: 256
Conv_103: 256
Conv_106: 256
Conv_108: 256
Conv_111: 512
Conv_113: 512
Conv_114: 512
Conv_117: 512
Conv_119: 512
Conv_122: 512
Conv_124: 512
Gemm_128: 512


In [62]:
pretrained_model=torch_model

**Integrating Pretrained model with Arcface Loss**

In [63]:
from collections import OrderedDict
import torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F

class PretrainedWithArcFace(nn.Module):
    def __init__(self, pretrained_model, feature_in=512, num_classes=540, margin=0.3, scale=64):
        super().__init__()
        self.backbone = pretrained_model
        self.arcface = ArcFace(feature_in=feature_in, feature_out=num_classes, margin=margin, scale=scale)

    def forward(self, x, target=None):
        # Get embeddings from pretrained model
        embeddings = self.backbone(x)

        # Normalize embeddings
        embeddings = F.normalize(embeddings, dim=1)

        if target is not None:
            # Training mode: pass through ArcFace to get logits
            logits = self.arcface(embeddings, target)
            return logits

        # Inference mode: just return normalized embeddings
        return embeddings


In [40]:
# Replace BatchNormalization_129 with BatchNorm1d(512)
"""pretrained_model.BatchNormalization_129 = torch.nn.BatchNorm1d(
    num_features=540,
    eps=1e-5,            # or match previous value: 9.999999747378752e-06
    momentum=0.1,        # or match previous: 0.10000002384185791
    affine=True,
    track_running_stats=True
)pretrained_model.Gemm_128=nn.Linear(pretrained_model.Gemm_128.in_features,540)
#class_weights = pretrained_model.BatchNormalization_129.weight.data
class_weights = pretrained_model.Gemm_128.weight.data

print(len(class_weights))"""


'pretrained_model.BatchNormalization_129 = torch.nn.BatchNorm1d(\n    num_features=540,\n    eps=1e-5,            # or match previous value: 9.999999747378752e-06\n    momentum=0.1,        # or match previous: 0.10000002384185791\n    affine=True,\n    track_running_stats=True\n)pretrained_model.Gemm_128=nn.Linear(pretrained_model.Gemm_128.in_features,540)\n#class_weights = pretrained_model.BatchNormalization_129.weight.data\nclass_weights = pretrained_model.Gemm_128.weight.data\n\nprint(len(class_weights))'

In [67]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PretrainedWithArcFace(
    pretrained_model=pretrained_model,  # your base model
    feature_in=512,
    num_classes=540,
    margin=0.3,
    scale=64
).to(DEVICE)
for name, module in model.named_modules():
    print(f"{name}: {module}")



: PretrainedWithArcFace(
  (backbone): GraphModule(
    (Conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (initializers): Module()
    (PRelu_1): OnnxPReLU()
    (BatchNormalization_2): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
    (Conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (PRelu_4): OnnxPReLU()
    (Conv_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (Conv_6): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
    (Add_7): OnnxBinaryMathOperation()
    (BatchNormalization_8): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
    (Conv_9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (PRelu_10): OnnxPReLU()
    (Conv_11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Add_12): OnnxBinaryMathOperation()
    (BatchNorm

In [68]:
# Choose how many layers to unfreeze
N = 6  # Last 6 layers

# Get all named parameters as a list
params = list(model.named_parameters())
print(len(params))
# Loop through with indexing
for idx, (name, param) in enumerate(params):
    if idx >= len(params) - N:
        param.requires_grad = True  # Enable training for last N layers
    else:
        param.requires_grad = False  # Freeze earlier layers


161


In [69]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [70]:
@torch.no_grad()
def accuracy(logits, targets):
    return (logits.argmax(1) == targets).float().mean().item()

def train_epoch(loader):
    model.train()
    total_loss, total_acc = 0.0, 0.0

    for X, y in loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()

        logits = model(X, target=y)
        loss = criterion(logits, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += accuracy(logits, y)

    n = len(loader)
    return total_loss / n, total_acc / n


In [71]:
EPOCHS = 5
for epoch in range(1, EPOCHS + 1):
    trn_loss, trn_acc = train_epoch(train_loader)
    print(f"Epoch {epoch:>2}: loss={trn_loss:.4f}, acc={trn_acc:.2%}")


Epoch  1: loss=11.9580, acc=51.50%
Epoch  2: loss=4.2515, acc=94.14%
Epoch  3: loss=4.1424, acc=94.53%
Epoch  4: loss=4.0615, acc=95.03%
Epoch  5: loss=4.0116, acc=95.31%


In [81]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        logits = model(X, target=y)  # Pass target to get logits
        # Get predicted class by selecting the index (class) with the highest logit score
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)


print('correct',correct)
print('total',total)
print(f"Test Accuracy: {correct / total:.2%}")


correct 38615
total 39539
Test Accuracy: 97.66%


540


In [42]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", DEVICE)
try:
    pretrained_model.to(DEVICE)
except Exception as e:
    print("Failed to move model to device:", e)


Using device: cuda


In [43]:
pretrained_model.to(device)  # Make sure this runs before any forward pass


GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initializers): Module()
  (PRelu_1): OnnxPReLU()
  (BatchNormalization_2): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_4): OnnxPReLU()
  (Conv_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Conv_6): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
  (Add_7): OnnxBinaryMathOperation()
  (BatchNormalization_8): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_10): OnnxPReLU()
  (Conv_11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Add_12): OnnxBinaryMathOperation()
  (BatchNormalization_13): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.

**Using oonx model**

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2



# Load the ONNX model
onnx_model_path = 'glint360k_r50.onnx'
ort_session = ort.InferenceSession(onnx_model_path)
for i, node in enumerate(onnx_model.graph.node):
    print(f"{i}: {node.op_type} - {node.name} - Outputs: {node.output}")
# Helper: preprocess image for InsightFace ONNX
def preprocess(img_path):
    img = cv2.imread(img_path)

    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")
    img = cv2.resize(img, (112, 112))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # ONNX expects RGB
    img = np.transpose(img, (2, 0, 1))          # (H, W, C) -> (C, H, W)
    img = img.astype(np.float32)
    img -= 127.5
    img /= 128.0
    img = np.expand_dims(img, axis=0)
    return img

# Prepare input
input1 = preprocess("0014_01.jpg")
input2 = preprocess("0007_01.jpg")

# Concatenate into batch
onnx_input = np.concatenate([input1, input2], axis=0)

# Get the actual input name of the model
input_name = ort_session.get_inputs()[0].name  # typically 'input.1'

# Run inference
onnx_output = ort_session.run(None, {input_name: onnx_input})

# Output embeddings (shape: [2, 512])
embeddings = onnx_output[0]
print("Embeddings shape:", embeddings.shape)

# Compute cosine similarity
def cosine_similarity(emb1, emb2):
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

def is_same_person(emb1, emb2, threshold=0.58):
    similarity = cosine_similarity(emb1, emb2)
    return  similarity > threshold

similarity_score = cosine_similarity(embeddings[0], embeddings[1])
is_same_person = is_same_person(embeddings[0], embeddings[1],threshold=0.6)
print(f"Similarity Score: {similarity_score:.4f}")
print(f"Same person? {'YES' if is_same_person else 'NO'}")


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Define path to target folder
base_path = "test_images"

# Get list of image file names
image_files = [f for f in os.listdir(base_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Sort (optional, for consistent order)
image_files.sort()

# Number of images to display
num_images = len(image_files)
cols = 15
rows = (num_images + cols - 1) // cols  # ceiling division

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(20, 2.5 * rows))
axes = axes.flatten()  # flatten in case rows > 1

for i in range(cols * rows):
    ax = axes[i]
    if i < num_images:
        img_path = os.path.join(base_path, image_files[i])
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img_rgb)
        ax.set_title(f"{i+1}", fontsize=8)
        ax.axis('off')
    else:
        ax.axis('off')  # hide unused subplots

plt.tight_layout()
plt.show()



In [ ]:
import os
import cv2
import numpy as np
import onnxruntime as ort

#  Load ONNX model ---
onx_model_path = 'glint360k_r50.onnx'
ort_session = ort.InferenceSession(onx_model_path)
input_name = ort_session.get_inputs()[0].name

#  Helper: preprocess image ---
def preprocess(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")
    img = cv2.resize(img, (112, 112))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1)).astype(np.float32)
    img = (img - 127.5) / 128.0
    return np.expand_dims(img, axis=0)

# Implementation of cosine similarity
def cosine_similarity(emb1, emb2):
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
#Implementation of a function for comparing two images and figure out whether or not those are similar to each other
def is_same_person(emb1, emb2, threshold=0.6):
    return cosine_similarity(emb1, emb2) > threshold

# Setting the chosen image path
img_chosen = "0014_01.jpg"
base_path = "test_images"
chosen_path = os.path.join(img_chosen)

# Get embedding of chosen image
input_chosen = preprocess(chosen_path)
embedding_chosen = ort_session.run(None, {input_name: input_chosen})[0][0]

#Iterating over the list of images in the directory
image_files = [f for f in os.listdir(base_path) if f.lower().endswith(('.jpg'))]
image_files.sort()

# Compare all images to the chosen one
similar_count = 0
threshold = 0.45
for image_file in image_files:
    if image_file == img_chosen:
        continue  # skip comparing to self
    img_path = os.path.join(base_path, image_file)

    input_img = preprocess(img_path)
    embedding_img = ort_session.run(None, {input_name: input_img})[0][0]
    sim_score = cosine_similarity(embedding_chosen, embedding_img)

    if sim_score > threshold:
        similar_count += 1
        print(f"{image_file}: Similarity = {sim_score:.4f}")
    else:
        print(f"{image_file}: Similarity = {sim_score:.4f}")



print(f"\nTotal similar images to '{img_chosen}' (threshold {threshold}): {similar_count}")



In [ ]:


folder_path = '//path'
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of files: {file_count}")
filename_to_folder